# Python Workflows

---

#### Best

- Python workflow package, validation, logging

## `snakemake` Introduction

Materials adapted from [Wolfgang Resch](https://hpc.nih.gov/training/handouts/170501_snakemake_class_materials.tgz), NIH

---

## snakemake

Reference: https://academic.oup.com/bioinformatics/article/28/19/2520/290322/Snakemake-a-scalable-bioinformatics-workflow

We Would Like a Tool That

- can figure out how to run a whole workflow based on a set of rules for transforming one file type to another 
- is reproducible
- reruns steps if necessary (input files change or processing steps change)
- runs any necessary steps automatically as new data is added

### Snakemake Is a Rule Based Dependency Tracker`

#### Rules

Rules describe how to transform one file type into another. Files are identified based on constant parts of their name (e.g. .fastq, _fastqc.zip, ...)

#### Dependancy Tracker

Snakemake automatically determines what files are needed to produce a certain file type based on the rules. This information is used to calculate a dependency tree for the whole workflow. Rules are only executed if their outputs either don't exist or are older than the input files.

There Are Many Such Tools
- make, ninja, scons, waf, ruffus, jug,
- Rake, bpipe, paver, Galaxy, ...

#### So Why Use `snakemake`?

- Snakefiles are python code - i.e. a real programming language is available
- designed with bioinformatics in mind
- easy to offload processes to cluster nodes
- advanced pattern matching
- multiple input and output files
- many bonus features: configuration, wrappers,
- target lists, graphs of workflow, reports, ...
- keeps track of code changes in rules

Chair of Bioinformatics Research Group, Boku University Vienna, Austria
"We have initially tested several systems, including, Bpipe [9], Moa [https://github.com/mfiers/Moa], Ruffus [28], and Snakemake [10]. We have since focused on exploring Snakemake due to, among other features, its make-like workflow definition, simple integration with Python, Bash code portability, ease of porting workflows to a cluster, intuitive parallelization, and ongoing active development. We are currently working on extending Snakemake with a lightweight modular system for development cycle control and policy-based specification of rules and requirements that supports an in-flow enforcement of consistency constraints. We have developed and validated a proof-of-concept prototype of the mechanism and automated the code generation of rules." [Source](https://biologydirect.biomedcentral.com/articles/10.1186/s13062-015-0071-8)

It is almost impossible to cover all available workflow management systems, but I can give you a few points: 
* compared to GNU Make, Snakemake is more flexible, supporting e.g. multiple output files, Python scripting, conditional inputs, arbitrary resource constraints, cluster execution with DRMAA. 
* Snakefiles can look very clean and readable (almost self documenting). 
* Snakemake allows you to easily separate workflow logic from analysis logic (in the form of e.g. external scripts), without having to write boilerplate code. This further supports readability and also code re-use. 
* We are building a library of Snakemake wrappers around popular bioinformatics tools. 
* Soon, Snakemake will support the automatic installation of the software dependencies of your workflow e.g. via Bioconda. This allows to deploy and execute a workflow on a new machine in a single step.

- [Johannes Köster](https://groups.google.com/forum/#!topic/snakemake/X_sGS6EiY-M)

### Biowulf example...

unsorted text file -> sorted text file

    rule sort:
     input: "words.txt"
     output: "words.sorted.txt"
     shell: "sort {input} > {output}"

- Rules start with the `rules` keyword

- Snakemake rule files are python and therefore whitespace sensitive

- Snakemake uses filenames to determine which rules to apply 

    rule sort:
     input: "words.txt"
     output: "words.sorted.txt"
     shell: "sort {input} > {output}"

- Rules can use shell:, python (run:), and R (run: R())

- Shell rules have to be quoted with single or triple quotes

- Use {...} to access input, output, parameters, threads, log, wildcards, and global variables. use {{...}} to get literal braces.

#### Rules With Wildcards

    rule sort:
     input: "{name}.txt"
     output: "{name}.sorted.txt"
     shell: "sort {input} > {output}

### Live Demo

In [ ]:
!cd ./170501_snakemake_class_materials/

In [ ]:
!file * | tr -s ':' ';' > files1.txt

In [ ]:
!ls -l | tr -s ' ' ';' > files2.txt

In [ ]:
%%writefile Snakefile
localrules: all, clean, merge

rule all:
    input: "files1-files2.txt"

rule merge:
    input: "{name1}.sorted.txt", 
           "{name2}.sorted.txt"
    output: "{name1}-{name2}.txt"
    shell: "join -t ';' -2 9 -o 1.1,2.1,1.2 -- {input} > {output}"

rule sort:
    input: "{name}.txt"
    output: "{name}.sorted.txt"
    log: "sort-%j.log"
    shell: "sort -k1,1 {input} > {output}"

rule clean:
    shell: "rm -f *.sorted.txt files1-files2.txt"

In [ ]:
!cat Snakefile

---

## Installing `snakemake`

In [ ]:
!conda install -y snakemake

---

In [ ]:
!snakemake -s Snakefile all

# since snakemake defaults to executing the first target in the
# rule file 'Snakefile' this is equivalent to
# !snakemake

#### Rerunnig snakemake if nothing changed is a no-op

In [ ]:
!snakemake

#### Clean up

In [ ]:
!snakemake clean

    -n, --dry-run: run without executing rules
    -p: print shell commands

In [ ]:
!snakemake clean
!snakemake -np

    --verbose: more information
    --reason: show reason why rule is being executed

    --list, -l: list all rules

In [ ]:
!snakemake clean
!snakemake -n --reason --verbose

In [ ]:
!snakemake clean
!snakemake --list

Use -S, --summary to show tabular information about all files generated by the workflow.

In [ ]:
!snakemake --summary

In [ ]:
!touch numbers1.txt
!snakemake --summary

In [ ]:
!snakemake -r

#### What happens when the code executed by a rule changes? Edit a rule and run

In [ ]:
!snakemake

Targets are not automatically recreated but code changes are tracked:

In [ ]:
!snakemake --lc # or --list-code-changes

#### Force recreation of all affected targets with

In [ ]:
!snakemake -R $(snakemake --lc)

#### Rules can be given explicit versions

In [ ]:
%%writefile Snakefile
localrules: all, clean, merge

rule all:
    input: "files1-files2.txt"

rule merge:
    input: "{name1}.sorted.txt", 
           "{name2}.sorted.txt"
    output: "{name1}-{name2}.txt"
    version: "1.0"
    shell: "join -t ';' -2 9 -o 1.1,2.1,1.2 -- {input} > {output}"

rule sort:
    input: "{name}.txt"
    output: "{name}.sorted.txt"
    log: "sort-%j.out"
    shell: "sort -k1,1 {input} > {output}"

rule clean:
    shell: "rm -f *.sorted.txt files1-files2.txt"

In [ ]:
!snakemake --lv # or --list-version-changes
!snakemake -R $(snakemake --lv)

### Visualize the workflow

In [ ]:
!conda install -y graphviz

In [ ]:
!snakemake --dag | dot -Tsvg > dag.svg

#### Param changes are tracked as well

In [ ]:
!snakemake --lp # or --list-param-changes
!snakemake -R $(snakemake --lp)

#### Running in Parallel - Locally

Snakemake workflows can be run in parallel on the local machine. -j specifies the number
of cpus snakemake is allowed to use. Snakemake will use the information from the threads section to determine how many jobs can be run at the same time.

## Resources:

__`snakemake`__
- http://snakemake.readthedocs.io/en/stable/tutorial/tutorial.html#tutorial
- [2017 snakemake tutorial](http://slides.com/johanneskoester/snakemake-tutorial-2016#/)Jeremy Leipzig, _[A review of bioinformaics pipeline frameworks](https://doi.org/10.1093/bib/bbw020)_. Briefings in Bioinformatics. 2016 
- Johannes Köster  Sven Rahmann, - [Snakemake—a scalable bioinformatics workflow engine](https://academic.oup.com/bioinformatics/article/28/19/2520/290322) Bioinformatics, Volume 28, Issue 19, 1 October 2012
- [snakemake wrappers](https://snakemake-wrappers.readthedocs.io/en/stable/index.html)
- [snakemake RNASeq example](https://github.com/seandavi/SnakemakeRNASeqExample)

- [Awesome Pipeline](https://github.com/pditommaso/awesome-pipeline] GitHub page